In [1]:

import numpy as np
import pandas as pd
from pymatgen.core.operations import SymmOp
from pymatgen.ext.matproj import MPRester
from jupyter_jsmol.pymatgen import quick_view
from jupyter_jsmol import JsmolView
from ipywidgets import Layout, widgets, interact
from pymatgen.core import Lattice, Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifWriter
import tempfile

In [2]:
mp_key = "jkj1S0TLmQtlvQmGDoUQvq5XQzLfKlCF"
mp_id = "mp-14437"

In [3]:
#je reprend mes valeurs du dernier devoir
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:

spaceAnalyzer = SpacegroupAnalyzer(prim_struc)
data = spaceAnalyzer.get_primitive_standard_structure()
data2 = spaceAnalyzer.get_conventional_standard_structure()
symopc= SpacegroupAnalyzer(data2).get_symmetry_operations()
symopp = SpacegroupAnalyzer(data).get_symmetry_operations()

inversion = symopc[1]
rotoinversion = symopc[7]
translation = symopc[13]
print(inversion.as_xyz_str())
print(rotation.as_xyz_str())
print(translation.as_xyz_str())
print(inversion)
print(rotation)
print(translation)

#


-x, -y, -z
-y, -x, z
-x+2/3, -y+1/3, -z+1/3
Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Rot:
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0.66666667 0.33333333 0.33333333]


In [5]:
view = quick_view(data2,"packed", conventional=True)
display(view)
print(data2.sites)
RB= data2.sites[0].frac_coords
Y = data2.sites[3].frac_coords
O = data2.sites[6].frac_coords
O2= data2.sites[9].frac_coords
list = [RB,Y,O,O2]
print("\n\n")
print(list)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

[PeriodicSite: Rb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0], PeriodicSite: Rb (1.741, -1.005, 6.509) [0.6667, 0.3333, 0.3333], PeriodicSite: Rb (1.741, 1.005, 13.02) [0.3333, 0.6667, 0.6667], PeriodicSite: Y (1.741, 1.005, 3.254) [0.3333, 0.6667, 0.1667], PeriodicSite: Y (3.481, -4.441e-16, 9.763) [1.0, 1.0, 0.5], PeriodicSite: Y (1.741, -1.005, 16.27) [0.6667, 0.3333, 0.8333], PeriodicSite: O (1.741, -1.005, 2.11) [0.6667, 0.3333, 0.108], PeriodicSite: O (0.0, 0.0, 4.399) [0.0, 0.0, 0.2253], PeriodicSite: O (1.741, 1.005, 8.619) [0.3333, 0.6667, 0.4414], PeriodicSite: O (1.741, -1.005, 10.91) [0.6667, 0.3333, 0.5586], PeriodicSite: O (0.0, 0.0, 15.13) [0.0, 0.0, 0.7747], PeriodicSite: O (1.741, 1.005, 17.42) [0.3333, 0.6667, 0.892]]



[array([0., 0., 0.]), array([0.33333333, 0.66666667, 0.16666667]), array([0.66666667, 0.33333333, 0.1080395 ]), array([0.66666667, 0.33333333, 0.55862716])]


In [6]:

IOUT = inversion.operate_multi(list)
print(IOUT)
print("\n")
RTOUT= rotation.operate_multi(list)
print(RTOUT)
print("\n")
TOUT = translation.operate_multi(list)
print(TOUT)


[[ 0.          0.          0.        ]
 [-0.33333333 -0.66666667 -0.16666667]
 [-0.66666667 -0.33333333 -0.1080395 ]
 [-0.66666667 -0.33333333 -0.55862716]]


[[ 0.          0.          0.        ]
 [-0.66666667 -0.33333333  0.16666667]
 [-0.33333333 -0.66666667  0.1080395 ]
 [-0.33333333 -0.66666667  0.55862716]]


[[ 0.66666667  0.33333333  0.33333333]
 [ 0.33333333 -0.33333333  0.16666667]
 [ 0.          0.          0.22529383]
 [ 0.          0.         -0.22529383]]


In [7]:
structure= data2.copy()

for site, new_coords in zip(structure.sites, IOUT):
    site.frac_coords = new_coords

script_inversion = f"""
draw sym1 [{list[1]}] color red;
draw sym2 [{IOUT[1]}] color blue;
"""
view_inversion = quick_view(structure, "packed", conventional=True)
view_inversion.script(script_inversion)
display(view_inversion)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [8]:
structure= data2.copy()
for site, new_coords in zip(structure.sites, TOUT):
    site.frac_coords = new_coords

script_translation = f"""
draw sym1 [{list[0]}] color red;
draw sym2 [{TOUT[0]}] color blue;
"""
view_translation = quick_view(structure, "packed", conventional=True)
view_translation.script(script_translation)
display(view_translation)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [9]:
structure= data2.copy()
for site, new_coords in zip(structure.sites, RTOUT):
    site.frac_coords = new_coords


script_rotation = f"""
draw sym1 [{list[2]}] color red;
draw sym2 [{RTOUT[2]}] color blue;
"""
view_rotation= quick_view(structure, "packed", conventional=True)
view_rotoinversion.script(script_rotation)
display(view_rotation)

JsmolView(layout=Layout(align_self='stretch', height='400px'))